Create Old Postring List

In [18]:
import re
import collections
import os
import time
import math
import random

In [19]:
indToDoc = {}
postingList = collections.defaultdict(list)
curDocID = 0
path = "practice_collection"

In [20]:
def tokenize(f, path):
    #Returns [(term, pos), (term, pos) ...]
    terms = []
    with open(path + "/" + f) as file:
        line = file.readline()
        i = 0
        while line:
            # Regex to match only strings and spaces 
            line = re.sub(r'[^A-Za-z\s]+', '', line)
            for word in line.split():
                terms.append((word.lower(), i))
                i += 1
            line = file.readline() 
    return terms

In [21]:
def createPostingList(path):
    startTime = time.time()
    for f in os.listdir(path):
        # TODO: remove in real code
        global curDocID
        
        indToDoc[curDocID] = f

        # Returns [(word, pos), (word, pos) ...]
        terms = tokenize(f, path)

        # create map {word:[pos1, pos2, pos3]}
        wordToPos = collections.defaultdict(list)
        for word, pos in terms:
            wordToPos[word].append(pos)

        # append to posting list {term : [(docID1, [pos1, pos2, pos3, pos4])]}
        for term, arr in wordToPos.items():
            postingList[term].append((curDocID, wordToPos[term]))

        # For every file update id 
        curDocID += 1
    
    endTime = time.time()
    print(f"Index built in {endTime - startTime} seconds.")
    return postingList

In [22]:
result = createPostingList(path)

Index built in 0.0009500980377197266 seconds.


Create New Postring List

In [23]:
newPostingList = collections.defaultdict(list)
idToTerm = {}
curTermIndex = 0

In [24]:
stopList = set()
stopTuples = tokenize("stop-list.txt", ".")
for t, p in stopTuples:
    stopList.add(t)
print(stopList)

{'as', 'the', 'to', 'by', 'of', 'in', 'on', 'it', 'a', 'its', 'at', 'will', 'that', 'is', 'and', 'were', 'are', 'an', 'for', 'from', 'be', 'has', 'was', 'he', 'with'}


In [25]:
for k,v in result.items():
    # if k in stopList:
    #     continue
    idToTerm[curTermIndex] = k
    newPostingList[curTermIndex].append(math.log10(len(indToDoc)/len(v)))
    
    for docId, posList in v:
        newPostingList[curTermIndex].append((docId, 1 + math.log10(len(posList)), posList))
    curTermIndex += 1

In [26]:
print(newPostingList)

defaultdict(<class 'list'>, {0: [0.17609125905568124, (0, 1.0, [0]), (1, 1.0, [0])], 1: [0.17609125905568124, (0, 1.0, [1]), (1, 1.0, [1])], 2: [0.47712125471966244, (0, 1.0, [2])], 3: [0.17609125905568124, (1, 1.0, [2]), (2, 1.0, [2])], 4: [0.47712125471966244, (2, 1.0, [0])], 5: [0.47712125471966244, (2, 1.0, [1])]})


Convert Each Document To A Vector

In [27]:
docToVec = {}
for k in indToDoc.keys():
    docToVec[k] = len(idToTerm)*[float(0)]

for k, v in newPostingList.items():
    idf = v[0]
    for i in range(1, len(v)):
        t = v[i]
        doc, w = t[0], t[1]
        docToVec[doc][k] = idf*w

In [28]:
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxx == 0 or sumyy == 0:
        return 0
    return sumxy/math.sqrt(sumxx*sumyy)

Only keep terms with heigher idf values

In [29]:
query_terms = ["a", "cat", "jumped"]
query_terms = [word.lower() for word in query_terms]

termToId = {}
for id, term in idToTerm.items():
    termToId[term] = id

termToIDF = []
for term in query_terms:
    if term in termToId:
        termToIDF.append((term, newPostingList[termToId[term]][0]))

termToIDF.sort(key=lambda x:x[1], reverse=True)
query_terms = [tuple[0] for tuple in termToIDF[:len(termToIDF)//2]] 

Create Query Vector by getting frequency of each term, multiplying it by idf of term, and putting it at index of term in query vector

In [30]:
termToFreq = {}
for t in query_terms:
    termToFreq[t] = termToFreq.get(t, 0) + 1

queryVector = len(idToTerm)*[float(0)]
for term, freq in termToFreq.items():
    if term in termToId:
        w = 1 + math.log10(freq)
        idf = newPostingList[termToId[term]][0]
        queryVector[termToId[term]] = w*idf

Get cosine similaity of each doc vector to the query vector

In [31]:
docToSimilarity = []
for doc, vec in docToVec.items(): 
    docToSimilarity.append((doc, cosine_similarity(queryVector, vec)))

In [34]:
docToSimilarity.sort(key = lambda x:x[1], reverse=True)
print([indToDoc[t[0]] for t in docToSimilarity[:2]])

['d3.txt', 'd1.txt']
